In [1]:
import scipy
import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import MeanShift, estimate_bandwidth

import pandas as pd

from scipy import stats
from scipy.stats import beta
from math import sin
from random import randint

import matplotlib.pyplot as plt
import itertools as it

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)

import collections

def recursively_default_dict():
        return collections.defaultdict(recursively_default_dict)



## Simulations

- Hein, J., Schierup, M., & Wiuf, C. (2004). Gene genealogies, variation and evolution: a primer in coalescent theory. Oxford University Press, USA. (*)

(*) referenced as GGVE throughout.


### Discrete and continuous coalescent - functions.

i. Geometric distribution.

- GGVE pp. 36 through 38.

In [2]:
from functools import reduce
from math import factorial
from operator import mul    # or mul=lambda x,y:x*y
from fractions import Fraction

def nCk(n,k):
  return int( reduce(mul, (Fraction(n-i, i+1) for i in range(k)), 1) )


def prob_noneF(k,Nsamp= 10):
    
    um= [1 - i / 2 / Nsamp for i in range(1, k)]
    
    um= reduce((lambda x, y: x * y), um)
    
    return um

def probOneF(k,Nsamp= 10):
    
    um= 1 - prob_noneF(k,Nsamp)
    
    return um

def CoalGeomF(j,k= 4,Nsamp= 10):
    '''
    Probability that (at least) 2 genes out of K find a common ancestor j generations ago.
    '''
    
    none_h= prob_noneF(k,Nsamp)
    prob_h= 1 - none_h
    
    um= (none_h)**(j-1) * prob_h
    
    return um


###
###

def probOne(k,Nsamp= 10):
    
    um= nCk(k,2) / 2 / Nsamp
    
    return um

def CoalGeom(j,k= 4,Nsamp= 10):
    '''
    Probability that (at least) 2 genes out of K find a common ancestor j generations ago.
    '''
    
    dt= 1 - probOne(k,Nsamp)
    dt= dt**(j-1)
    dt= dt * probOne(k,Nsamp)
    
    return dt

def CoalGeom_mean(j,mean= 2.5):
    '''
    Probability that (at least) 2 genes out of K find a common ancestor j generations ago.
    '''
    prob= 1 / mean
    
    dt= 1 - prob
    dt= dt**(j-1)
    dt= dt * prob
    
    return dt


ii. Exponential Distribution.

- GGVE pp. 39.

In [3]:
def expTk_cdf(t,k= 4):
    
    power= (-1) * nCk(k,2) * t
    um= 1 - np.exp(power)
    
    return um


def expTk_pdf(t,k= 4):
        
    a= nCk(k,2)
    
    um= a * np.exp(-a * t)
    
    return um


def tkdens_mean(t,meanA= 2.5):
    
    a= 1 / meanA
    
    um= a * np.exp(-a * t)
    
    return um


**plot**

In [4]:
Nsamp= 10
k= 4
example_mean= 2.5


In [5]:

X_geom= list(range(11))
X_exp= list(np.linspace(0,10,50))


geomF= [CoalGeomF(x,k= k,Nsamp=Nsamp) for x in X_geom]
geomS= [CoalGeom(x,k= k,Nsamp=Nsamp) for x in X_geom]
geomM= [CoalGeom_mean(x,mean=example_mean) for x in X_geom]

Exp_cdf= [expTk_cdf(x,k= k) for x in X_exp]
Exp_pdf= [expTk_pdf(x,k= k) for x in X_exp]
Exp_mean= [tkdens_mean(x,meanA= example_mean) for x in X_exp]

fig_c= [go.Scatter(
    x= X_geom,
    y= geomF,
    mode= 'lines',
    name= 'Geom full'
)]

fig_geom2= [go.Scatter(
    x= X_geom,
    y= geomS,
    mode= 'lines',
    name= 'Geom Simp'
)]

fig_geom_mean= [go.Scatter(
    x= X_geom,
    y= geomS,
    mode= 'lines',
    name= 'Geom X= {}'.format(example_mean)
)]


fig_exp_II= [go.Scatter(
    x= X_exp,
    y= Exp_mean,
    mode= 'lines',
    name= 'Exp dens X= {}'.format(example_mean)
)]


fig_c.extend(fig_geom2)
fig_c.extend(fig_exp_II)
fig_c.extend(fig_geom_mean)



layout= go.Layout(
    title= 'Discrete and continuous coalescent. k= {}; n= {} if not mean.'.format(k,Nsamp),
    yaxis= dict(title= 'P'),
    xaxis= dict(title= 't')
)


Fig= go.Figure(data= fig_c, layout= layout)
iplot(Fig)

In [6]:
fig_exp= []

##
fig_exp_cdf= [go.Scatter(
    x= X_exp,
    y= Exp_cdf,
    mode= 'lines',
    name= '1 - exp(-at)'
)]

fig_exp_pdf= [go.Scatter(
    x= X_exp,
    y= Exp_pdf,
    mode= 'lines',
    name= 'a * exp(-at)'
)]

fig_exp.extend(fig_exp_cdf)
fig_exp.extend(fig_exp_pdf)

layout= go.Layout(
    title= 'dist of Tck for k -> k-1; k= {} n= {}.'.format(k,Nsamp),
    yaxis= dict(title= 'P'),
    xaxis= dict(title= 't')
)


Fig= go.Figure(data= fig_exp, layout= layout)
iplot(Fig)

#### Algorithm I.

Simple coalescence.

- GGVE pp. 39.

In [74]:
from random import choices
from scipy.stats import expon
   
def get_time(k= 4,Nt=10):
    
    data= expon.rvs(size= Nt,scale= 1 / nCk(k,2))
    
    return data


def sim_coales(k= 4):
    
    Tcs= []
    
    for co in range(k-1):
        
        td= get_time(k= k - co,Nt=1)
        
        Tcs.extend(td)
        
        
    return Tcs


def SimI_net(k= 4):

    times= sim_coales(k= k)
    times_acc= [sum(times[:(x+1)]) for x in range(len(times))]
    #print(times_acc)
    #times_acc= times_acc[::-1]
    

    node_ord= np.random.choice(list(range(k)), k, replace= False)

    sim_keys= {z: [] for z in range(k)}
        
    edges= []
    times_dict= {}
    
    leaves= {z: [z] for z in range(k)}
    
    surface= list(range(k))
    #random.shuffle(surface)
    
    for cl in range(len(times)):
        
        if cl == len(times) - 1:
            new_nd= -1
        
        else:
            new_nd= cl + k
        
        pair_idx= np.random.choice(list(range(len(surface))),2,replace= False)
        
        pair= tuple([surface[x] for x in pair_idx])
        
        new_edges= [tuple([new_nd,x]) for x in pair]
        edges.extend(new_edges)
        
        surface= [surface[x] for x in range(len(surface)) if x not in pair_idx]        
        surface.append(new_nd)
        
        sim_keys[new_nd]= list(pair)
        
        
        
        ti= round(times_acc[cl],2)
        times_dict[new_nd]= ti
        leaves[new_nd]= ['t: {}'.format(ti)]
        
    
    return sim_keys, times_acc, leaves, edges, times_dict



In [76]:
from structure_tools.Coalesce_plots import plot_phyl_net

k= 8

sim_keys, times_acc, leaves, edges, times_dict= SimI_net(k= k)

##
node_list= list(sim_keys.keys())
root= True
nodes_as_seqs= False

plot_phyl_net([],leaves,node_list,edges,
              nodes_as_seqs= nodes_as_seqs,root= root)


### Quantities

#### i. Height of a tree

In [9]:


def prob_Hn(t,Nsamp):
    
    void= 0
    
    for k in range(1,Nsamp):
        
        nb= [Nsamp - x for x in range(k)]
        nb= reduce((lambda x, y: x * y), nb)

        ap= [Nsamp + x for x in range(k)]
        ap= reduce((lambda x, y: x * y), ap)
        
        um= (-1)**(k-1) * (2*k - 1) * nb
        um= um / ap
        um= np.exp(-nCk(k,2) * t) * um
        
        void += um
    
    return void


def Hmean(NSamp):
    um= 2 * (1 - 1 / Nsamp)
    
    return um


def Varmean(Nsamp):
    
    void= 0
    
    um= [1 / (j**2 * (j-1)**2) for j in range(2,Nsamp)]
    um= sum(um) * 4
    
    return um



#### ii. Length of a tree

In [10]:
def LnT(t,Nsamp):
    
    um= (1 - np.exp(-t/2))**(Nsamp-1)
    
    return um


def meanLt(Nsamp):
    
    um= 2 * sum([1 / x for x in range(1,Nsamp)])
    
    return um


def varLn(Nsamp):
    
    um= 4 * sum([1 / j**2 for j in range(1,Nsamp)])
    
    return um

In [11]:

X_exp= list(np.linspace(0,10,50))

Hm= [prob_Hn(t,k) for t in X_exp]
mean_h= Hmean(k)
var_h= Varmean(k)

Lm= [LnT(t,k) for t in X_exp]
mean_l= meanLt(k)
var_l= varLn(k)

###

fig_c= [go.Scatter(
    x= X_exp,
    y= Hm,
    mode= 'lines',
    name= 'H P(Ht <= x)'
)]

fig_c.append(go.Scatter(
    x= X_exp,
    y= Lm,
    mode= 'lines',
    name= 'L P(Tl <= x)'
))

layout= go.Layout(
    title= 'Average tree height and length for k= {}. Var= {}'.format(k,round(var_h,3)),
    yaxis= dict(title= 'P'),
    xaxis= dict(title= 't'),
    shapes= [
        # Line Vertical
        {
            'type': 'line',
            'x0': mean_h,
            'y0': 0,
            'x1': mean_h,
            'y1': max(Hm),
            'line': {
                'color': 'rgb(30,144,255)',
                'width': 3,
            },
        },
            {
            'type': 'line',
            'x0': mean_l,
            'y0': 0,
            'x1': mean_l,
            'y1': max(Lm),
            'line': {
                'color': 'rgb(255,140,0)',
                'width': 3,
            },
        }]
)


Fig= go.Figure(data= fig_c, layout= layout)
iplot(Fig)

### Algorithms for simulating sequence evolution.

i. Algorithm I.

- GGVE pp. 56

In [64]:
Nsamp= 10
Theta= 2

def get_time_II(k= 4,Theta= 1,Nt=10):
    
    par= k * (k - 1 + Theta) / 2
    
    data= expon.rvs(size= Nt,scale= 1 / par)
    
    return data


def toincoss(k= 4,Theta= 1):
    
    coal= (k - 1) / (k - 1 + Theta)
    
    probs= [1- coal, coal]
    
    head= np.random.choice([0,1],1,p= probs)
    
    return head


def SimII_net(k= 4,Theta= 1):

    sim_keys= {z: [] for z in range(k)}
        
    edges= []
    
    leaves= {z: [z] for z in range(k)}
    
    surface= list(range(k))
    
    time_k= 0
    
    cl= k
    d= 0
    
    while len(surface) >= 1:
        
        if len(surface) == 1:
            new_nd= -1
        
        else:
            new_nd= k + d
        
        timez= get_time_II(k= cl,Theta= Theta, Nt= 1)[0]
        time_k += timez
        
        coin= toincoss(k= cl, Theta= Theta)
        
        if coin == 1:
            
            if len(surface) == 2:
                new_nd= -1
            
            pair_idx= np.random.choice(list(range(len(surface))),2,replace= False)

            pair= tuple([surface[x] for x in pair_idx])

            new_edges= [tuple([new_nd,x]) for x in pair]
            edges.extend(new_edges)

            surface= [surface[x] for x in range(len(surface)) if x not in pair_idx]        
            surface.append(new_nd)

            sim_keys[new_nd]= list(pair)

            ti= round(time_k,2)
            leaves[new_nd]= ['coal. t: {}'.format(ti)]
        
        else:
            
            idx_mut= np.random.choice(list(range(len(surface))),1)[0]
            
            who_mut= surface[idx_mut]
            
            new_edge= (new_nd,who_mut)
            
            edges.append(new_edge)
            
            surface= [surface[x] for x in range(len(surface)) if x != idx_mut]
            surface.append(new_nd)
            
            sim_keys[new_nd]= [who_mut]
            
            ti= round(time_k,2)
            leaves[new_nd]= ['mut. t: {}'.format(ti)]
        
        if new_nd==-1:
            surface= []
        
        cl = len(surface)
        d += 1
            
    
    return sim_keys, times_acc, leaves, edges



In [159]:
k= 6
Theta= 2.04

sim_keys, times_acc, leaves, edges= SimII_net(k= k, Theta= Theta)

##
node_list= list(sim_keys.keys())
root= True
nodes_as_seqs= False

plot_phyl_net([],leaves,node_list,edges,
              nodes_as_seqs= nodes_as_seqs,root= root)


ii. Algorithm III

- GGVE pp. 57

Begin with algirithm I (above). Then incorporate mutations per branch at a rate ~ Pois(t * Theta / 2).

In [152]:


def SimIII(k= 4, Theta= 1):
    sim_keys, times_acc, leaves, edges, times_dict= SimI_net(k= k)

    proxy_edges= list(edges)

    for ed in proxy_edges:

        t0= times_dict[ed[0]]

        if ed[1] in times_dict.keys():
            t1= times_dict[ed[1]]
        else:
            t1= 0

        tt= t0 - t1

        Pexp= tt * Theta / 2

        muts= np.random.poisson(Pexp,1)[0]

        if muts > 0:

            curDictL= len(sim_keys)

            new_dict= {}
            new_edges= []
            new_leaves= {}

            for z in range(curDictL,curDictL + muts - 1):
                new_dict[z]= [z+1]
                new_leaves[z]= ''
                new_edges.append((z,z+1))


            new_dict.update({curDictL+muts-1: [ed[1]]})

            new_edges.append((ed[0],curDictL))
            new_edges.append((curDictL+muts-1,ed[1]))

            sim_keys[ed[0]]= [x for x in sim_keys[ed[0]] if x != ed[1]]
            sim_keys[ed[0]].append(curDictL)

            edges= [x for x in edges if x != ed]

            leaves.update(new_leaves)
            sim_keys.update(new_dict)
            edges.extend(new_edges)
    
    return sim_keys, leaves, edges



In [158]:
k= 6
Theta= 2.04

sim_keys, leaves, edges= SimIII(k= k, Theta= Theta)


node_list= list(sim_keys.keys())
plot_phyl_net([],leaves,node_list,edges,
              nodes_as_seqs= nodes_as_seqs,root= root)
